<a href="https://colab.research.google.com/github/Sabri-Nang/analisis_datos/blob/version1/analisis_estadisticas_criminales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Datos extraídos de:
[datos.gob.ar](https://www.datos.gob.ar/dataset/seguridad-snic---departamental-estadisticas-criminales-republica-argentina-por-departamentos/archivo/seguridad_3.2)

In [2]:
ruta_csv = '/content/snic-departamentos-anual.csv'

In [3]:
# importo las librerías
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [69]:
df = pd.read_csv(ruta_csv, delimiter=';')


In [8]:
# muestra
df.sample(5)

,provincia_id,provincia_nombre,departamento_id,departamento_nombre,anio,codigo_delito_snic_id,cod_delito,codigo_delito_snic_nombre,cantidad_hechos,cantidad_victimas,cantidad_victimas_masc,cantidad_victimas_fem,cantidad_victimas_sd,tasa_hechos,tasa_victimas,tasa_victimas_masc,tasa_victimas_fem
288038,42,La Pampa,42002,Norte (General Pico),2005,22,22,Delitos contra la seguridad pública,2,NaN,NaN,NaN,NaN,1.593067,NaN,NaN,NaN
34412,6,Buenos Aires,6154,Carlos Tejedor,2000,25,25,Delitos contra los poderes públicos y el orden...,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
312494,50,Mendoza,50014,General Alvear,2000,21,21,Otros delitos contra la propiedad,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
464303,86,Santiago del Estero,86098,Juan F. Ibarra,2022,4,4,Homicidios culposos por otros hechos,0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
28403,6,Buenos Aires,6105,Bolívar,2016,3,3,Muertes en accidentes viales,7,7.0,7.0,0.0,0.0,19.777922,19.777922,40.038895,0.0


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497037 entries, 0 to 497036
Data columns (total 17 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   provincia_id               497037 non-null  int64  
 1   provincia_nombre           497037 non-null  object 
 2   departamento_id            497037 non-null  int64  
 3   departamento_nombre        497037 non-null  object 
 4   anio                       497037 non-null  int64  
 5   codigo_delito_snic_id      497037 non-null  object 
 6   cod_delito                 497037 non-null  int64  
 7   codigo_delito_snic_nombre  497037 non-null  object 
 8   cantidad_hechos            497037 non-null  int64  
 9   cantidad_victimas          176027 non-null  float64
 10  cantidad_victimas_masc     176027 non-null  float64
 11  cantidad_victimas_fem      176027 non-null  float64
 12  cantidad_victimas_sd       176027 non-null  float64
 13  tasa_hechos                47

In [14]:
df.shape

(497037, 17)

In [15]:
df.columns

Index(['provincia_id', 'provincia_nombre', 'departamento_id',
       'departamento_nombre', 'anio', 'codigo_delito_snic_id', 'cod_delito',
       'codigo_delito_snic_nombre', 'cantidad_hechos', 'cantidad_victimas',
       'cantidad_victimas_masc', 'cantidad_victimas_fem',
       'cantidad_victimas_sd', 'tasa_hechos', 'tasa_victimas',
       'tasa_victimas_masc', 'tasa_victimas_fem'],
      dtype='object')

In [70]:
# veamos si codigo_delito_snic_id	 es igual a cod_delito

df['codigo_delito_snic_id'].astype(pd.Int64Dtype()).isnull().sum()

0

In [71]:
(df['codigo_delito_snic_id'].astype(pd.Int64Dtype()) != df['cod_delito']).sum() # ambas columnas son iguales

0

In [72]:
len(df['cod_delito'].unique()) # cantidad de codigos distintos

72

In [73]:
len(df['codigo_delito_snic_nombre'].unique())

72

In [74]:
# quiero ver si cada cod_delito corresponde a un codigo_delito_snic_nombre, o hay mas de uno para el mismo codigo
df.groupby('cod_delito')['codigo_delito_snic_nombre'].nunique().reset_index()

,cod_delito,codigo_delito_snic_nombre
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1
...,...,...
67,295,2
68,296,2
69,297,1
70,298,2


In [75]:
# veamos los casos que un codigo engloba mas de un delito
codigos_delitos = df.groupby('cod_delito')['codigo_delito_snic_nombre'].unique().reset_index()

In [76]:
codigos_delitos['cantidad_delitos'] = codigos_delitos['codigo_delito_snic_nombre'].apply(len)

In [77]:
codigos_delitos

,cod_delito,codigo_delito_snic_nombre,cantidad_delitos
0,1,[Homicidios dolosos],1
1,2,[Homicidios dolosos en grado de tentativa],1
2,3,[Muertes en accidentes viales],1
3,4,[Homicidios culposos por otros hechos],1
4,5,[Lesiones dolosas],1
...,...,...,...
67,295,"[Contrabando Simple, Contrabando simple]",2
68,296,"[Contrabando Agravado, Contrabando agravado]",2
69,297,[Contrabando de elementos nucleares agresivos ...,1
70,298,"[Otros delitos previstos en leyes especiales, ...",2


In [78]:
distintos = codigos_delitos[codigos_delitos['cantidad_delitos'] > 1]
distintos

,cod_delito,codigo_delito_snic_nombre,cantidad_delitos
16,17,[Robos agravados por el resultado de lesiones ...,2
67,295,"[Contrabando Simple, Contrabando simple]",2
68,296,"[Contrabando Agravado, Contrabando agravado]",2
70,298,"[Otros delitos previstos en leyes especiales, ...",2


In [79]:
# hay diferencias en los formatos, espacios, mayusculas
# formateo sacando espacios y cambiando todo a minuscula
df['codigo_delito_snic_nombre'] = df['codigo_delito_snic_nombre'].str.strip()

In [80]:
df['codigo_delito_snic_nombre'] = df['codigo_delito_snic_nombre'].str.lower()

In [87]:
# verifico
codigos_delitos = df.groupby('cod_delito')['codigo_delito_snic_nombre'].unique().reset_index()
codigos_delitos['cantidad_delitos'] = codigos_delitos['codigo_delito_snic_nombre'].apply(len)
codigos_delitos[codigos_delitos['cantidad_delitos'] > 1]

,cod_delito,codigo_delito_snic_nombre,cantidad_delitos


In [83]:
codigos_delitos[codigos_delitos['cantidad_delitos'] > 1]['codigo_delito_snic_nombre'].iloc[0]

array(['otros delitos previstos en leyes especiales',
       'otros delitos previstos en leyes especiales n.c.p'], dtype=object)

In [86]:
# reemplazo 'otros delitos previstos en leyes especiales n.c.p' por 'otros delitos previstos en leyes especiales'
df['codigo_delito_snic_nombre'] = df['codigo_delito_snic_nombre'].replace('otros delitos previstos en leyes especiales n.c.p', 'otros delitos previstos en leyes especiales')

In [90]:
# distintas provincias
print(len(df['provincia_nombre'].unique()))
print(len(df['provincia_id'].unique()))

24
24


In [91]:
df['provincia_nombre'].unique()

array(['Ciudad Autónoma de Buenos Aires', 'Buenos Aires', 'Catamarca',
       'Córdoba', 'Corrientes', 'Chaco', 'Chubut', 'Entre Ríos',
       'Formosa', 'Jujuy', 'La Pampa', 'La Rioja', 'Mendoza', 'Misiones',
       'Neuquén', 'Río Negro', 'Salta', 'San Juan', 'San Luis',
       'Santa Cruz', 'Santa Fe', 'Santiago del Estero', 'Tucumán',
       'Tierra del Fuego, Antártida e Islas del Atlántico Sur'],
      dtype=object)

In [94]:
# como variaron los delitos por año
# agrupo por año y delito
df.groupby(['anio', 'cod_delito', 'codigo_delito_snic_nombre']).aggregate({'cantidad_hechos': 'sum'})

cantidad_hechos
anio cod_delito codigo_delito_snic_nombre                                          
2000 1          homicidios dolosos                                             2653
     2          homicidios dolosos en grado de tentativa                        884
     3          muertes en accidentes viales                                   3686
     4          homicidios culposos por otros hechos                           1058
     5          lesiones dolosas                                             107864
...                                                                             ...
2023 295        contrabando simple                                            17898
     296        contrabando agravado                                            448
     297        contrabando de elementos nucleares agresivos qu...                7
     298        otros delitos previstos en leyes especiales                   40778
     2810       otros delitos previstos en la ley 23.737                       7432

[938 rows x 1 columns]

In [98]:
anios_cantidad = df.groupby(['anio', 'cod_delito', 'codigo_delito_snic_nombre']).aggregate({'cantidad_hechos': 'sum'}).reset_index()
anios_cantidad

,anio,cod_delito,codigo_delito_snic_nombre,cantidad_hechos
0,2000,1,homicidios dolosos,2653
1,2000,2,homicidios dolosos en grado de tentativa,884
2,2000,3,muertes en accidentes viales,3686
3,2000,4,homicidios culposos por otros hechos,1058
4,2000,5,lesiones dolosas,107864
...,...,...,...,...
933,2023,295,contrabando simple,17898
934,2023,296,contrabando agravado,448
935,2023,297,contrabando de elementos nucleares agresivos q...,7
936,2023,298,otros delitos previstos en leyes especiales,40778


In [100]:
maximos_delitos_anuales = anios_cantidad.groupby('anio')['cantidad_hechos'].max().reset_index()
maximos_delitos_anuales

,anio,cantidad_hechos
0,2000,320649
1,2001,345725
2,2002,423478
3,2003,393547
4,2004,352554
5,2005,322267
6,2006,318613
7,2007,331723
8,2008,361716
9,2009,367975


In [101]:
# merge para obtener delitos
pd.merge(anios_cantidad, maximos_delitos_anuales, on=['anio', 'cantidad_hechos'])

,anio,cod_delito,codigo_delito_snic_nombre,cantidad_hechos
0,2000,15,robos (excluye los agravados por el resultado ...,320649
1,2001,15,robos (excluye los agravados por el resultado ...,345725
2,2002,15,robos (excluye los agravados por el resultado ...,423478
3,2003,15,robos (excluye los agravados por el resultado ...,393547
4,2004,15,robos (excluye los agravados por el resultado ...,352554
5,2005,15,robos (excluye los agravados por el resultado ...,322267
6,2006,15,robos (excluye los agravados por el resultado ...,318613
7,2007,15,robos (excluye los agravados por el resultado ...,331723
8,2008,15,robos (excluye los agravados por el resultado ...,361716
9,2009,15,robos (excluye los agravados por el resultado ...,367975


In [108]:
df['codigo_delito_snic_nombre'].unique()

array(['homicidios dolosos',
       'abusos sexuales con acceso carnal (violaciones)',
       'otros delitos contra la integridad sexual',
       'delitos contra el estado civil', 'amenazas',
       'otros delitos contra la libertad',
       'robos (excluye los agravados por el resultado de lesiones y/o muertes)',
       'tentativas de robo (excluye las agravadas por el res. de lesiones y/o muerte)',
       'robos agravados por el resultado de lesiones y/o muertes',
       'tentativas de robo agravado por el resultado de lesiones y/o muertes',
       'hurtos', 'homicidios dolosos en grado de tentativa',
       'tentativas de hurto', 'otros delitos contra la propiedad',
       'delitos contra la seguridad pública',
       'delitos contra el orden público',
       'delitos contra la seguridad de la nación',
       'delitos contra los poderes públicos y el orden constitucional',
       'delitos contra la administración pública',
       'delitos contra la fe pública', 'ley 23.737 (estupefa

In [112]:
# analizo accidentes viales
accidentes_viales = df[df['codigo_delito_snic_nombre'] == 'muertes en accidentes viales']
accidentes_viales.sample(5)

,provincia_id,provincia_nombre,departamento_id,departamento_nombre,anio,codigo_delito_snic_id,cod_delito,codigo_delito_snic_nombre,cantidad_hechos,cantidad_victimas,cantidad_victimas_masc,cantidad_victimas_fem,cantidad_victimas_sd,tasa_hechos,tasa_victimas,tasa_victimas_masc,tasa_victimas_fem
14143,2,Ciudad Autónoma de Buenos Aires,2999,Departamento sin determinar,2006,3,3,muertes en accidentes viales,119,119.0,82.0,29.0,8.0,NaN,NaN,NaN,NaN
471366,86,Santiago del Estero,86154,Rivadavia,2013,3,3,muertes en accidentes viales,0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0000
220293,22,Chaco,22105,9 de Julio,2006,3,3,muertes en accidentes viales,6,7.0,5.0,2.0,0.0,21.229919,24.768240,NaN,NaN
426763,78,Santa Cruz,78014,Deseado,2018,3,3,muertes en accidentes viales,8,9.0,7.0,2.0,0.0,5.682545,6.392863,9.679605,2.9212
139600,6,Buenos Aires,6882,Zárate,2013,3,3,muertes en accidentes viales,14,14.0,0.0,0.0,14.0,11.756703,11.756703,0.000000,0.0000


In [114]:
# agrupo por provincia y año
accidentes_viales.value_counts()

provincia_id  provincia_nombre                                       departamento_id  departamento_nombre  anio  codigo_delito_snic_id  cod_delito  codigo_delito_snic_nombre     cantidad_hechos  cantidad_victimas  cantidad_victimas_masc  cantidad_victimas_fem  cantidad_victimas_sd  tasa_hechos  tasa_victimas  tasa_victimas_masc  tasa_victimas_fem
2             Ciudad Autónoma de Buenos Aires                        2001             Comuna 1             2010  3                      3           muertes en accidentes viales  0                0.0                0.0                     0.0                    0.0                   0.000000     0.000000       0.000000            0.000000             1
50            Mendoza                                                50119            Tunuyán              2017  3                      3           muertes en accidentes viales  4                4.0                4.0                     0.0                    0.0                   6.953498     6.953498       13.921760           0.000000             1
                                                                     50126            Tupungato            2014  3                      3           muertes en accidentes viales  6                6.0                6.0                     0.0                    0.0                   16.890942    16.890942      33.070606           0.000000             1
                                                                                                           2013  3                      3           muertes en accidentes viales  9                9.0                8.0                     1.0                    0.0                   25.760654    25.760654      44.820438           5.852060             1
                                                                                                           2012  3                      3           muertes en accidentes viales  6                6.0                5.0                     1.0                    0.0                   17.469284    17.469284      28.486782           5.954507             1
                                                                                                                                                                                                                                                                                                                                                               ..
14            Córdoba                                                14028            Cruz del Eje         2023  3                      3           muertes en accidentes viales  12               14.0               10.0                    4.0                    0.0                   16.756266    19.548977      27.867573           11.194761            1
                                                                                                           2022  3                      3           muertes en accidentes viales  11               11.0               11.0                    0.0                    0.0                   15.573678    15.573678      31.103319           0.000000             1
                                                                                                           2021  3                      3           muertes en accidentes viales  6                6.0                4.0                     2.0                    0.0                   8.615120     8.615120       11.478750           5.747457             1
                                                                                                           2020  3                      3           muertes en accidentes viales  6                6.0                4.0                     2.0                    0.0                   8.739222     8.739222       11.652635           5.825978             1
94            Tierra del Fuego, Antártida e Islas del Atlántico Sur  94014            Ushuaia              2023